In [69]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   --------------------- ------------------ 0.8/1.5 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 6.9 MB/s eta 0:00:00


In [70]:
import numpy as np
import matplotlib as mp
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import lightgbm as lgb

Let's import the data

In [5]:
df = pd.read_csv("bank_train.csv")

In [6]:
df.head()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593994 entries, 0 to 593993
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    593994 non-null  int64  
 1   annual_income         593994 non-null  float64
 2   debt_to_income_ratio  593994 non-null  float64
 3   credit_score          593994 non-null  int64  
 4   loan_amount           593994 non-null  float64
 5   interest_rate         593994 non-null  float64
 6   gender                593994 non-null  object 
 7   marital_status        593994 non-null  object 
 8   education_level       593994 non-null  object 
 9   employment_status     593994 non-null  object 
 10  loan_purpose          593994 non-null  object 
 11  grade_subgrade        593994 non-null  object 
 12  loan_paid_back        593994 non-null  float64
dtypes: float64(5), int64(2), object(6)
memory usage: 58.9+ MB


In [27]:
print("Class balance:")
print(df['loan_paid_back'].value_counts())
print("\nPercentages:")
print(df['loan_paid_back'].value_counts(normalize=True) * 100)

Class balance:
loan_paid_back
1.0    474494
0.0    119500
Name: count, dtype: int64

Percentages:
loan_paid_back
1.0    79.881952
0.0    20.118048
Name: proportion, dtype: float64


Group and check correlations

Correlation between Annual Income and Repayment

In [8]:
df.groupby('loan_paid_back')['annual_income'].mean()

loan_paid_back
0.0    47875.503881
1.0    48296.999711
Name: annual_income, dtype: float64

Correlation between Credit Score and Repayment

In [13]:
df.groupby('loan_paid_back')['credit_score'].mean()

loan_paid_back
0.0    655.010628
1.0    687.440208
Name: credit_score, dtype: float64

Cross Tabulation Between Grade of Loan and payback

In [14]:
pd.crosstab(df['grade_subgrade'], df['loan_paid_back'], normalize='index') * 100

loan_paid_back,0.0,1.0
grade_subgrade,,
A1,4.750000,95.250000
A2,4.707631,95.292369
A3,4.453049,95.546951
A4,4.291593,95.708407
A5,5.503845,94.496155
B1,8.365867,91.634133
B2,6.257005,93.742995
B3,5.995979,94.004021
B4,6.824242,93.175758


Cross Tabulation between Loan_Purpose and Loan Paid Back

In [21]:
pd.crosstab(index=[df['employment_status'], df['loan_purpose']], columns=df['loan_paid_back'])

loan_paid_back                          0.0     1.0
employment_status loan_purpose                     
Employed          Business             2761   24465
                  Car                  4573   38975
                  Debt consolidation  26324  220529
                  Education            3186   24036
                  Home                 3326   30953
                  Medical              1929   14744
                  Other                4932   43419
                  Vacation              672    5821
Retired           Business                1    1068
                  Car                     8    1736
                  Debt consolidation     28    8600
                  Education               2    1015
                  Home                    6    1322
                  Medical                 1     642
                  Other                   0    1844
                  Vacation                0     180
Self-employed     Business              283    2703
                  Car                   532    4997
                  Debt consolidation   2865   25315
                  Education             385    2947
                  Home                  350    3378
                  Medical               253    2062
                  Other                 602    5138
                  Vacation               59     611
Student           Business              501     202
                  Car                   885     329
                  Debt consolidation   4751    1667
                  Education             629     188
                  Home                  635     306
                  Medical               393     114
                  Other                 885     302
                  Vacation              108      36
Unemployed        Business             3052     267
                  Car                  5587     486
                  Debt consolidation  31974    2642
                  Education            3967     286
                  Home                 3482     360
                  Medical              2485     183
                  Other                6204     548
                  Vacation              884      78

In [18]:
pd.crosstab(index=[df['employment_status'], df['loan_purpose']], columns=df['loan_paid_back'], normalize='index')*100

loan_paid_back                              0.0         1.0
employment_status loan_purpose                             
Employed          Business            10.141042   89.858958
                  Car                 10.501056   89.498944
                  Debt consolidation  10.663836   89.336164
                  Education           11.703769   88.296231
                  Home                 9.702733   90.297267
                  Medical             11.569604   88.430396
                  Other               10.200410   89.799590
                  Vacation            10.349607   89.650393
Retired           Business             0.093545   99.906455
                  Car                  0.458716   99.541284
                  Debt consolidation   0.324525   99.675475
                  Education            0.196657   99.803343
                  Home                 0.451807   99.548193
                  Medical              0.155521   99.844479
                  Other                0.000000  100.000000
                  Vacation             0.000000  100.000000
Self-employed     Business             9.477562   90.522438
                  Car                  9.621993   90.378007
                  Debt consolidation  10.166785   89.833215
                  Education           11.554622   88.445378
                  Home                 9.388412   90.611588
                  Medical             10.928726   89.071274
                  Other               10.487805   89.512195
                  Vacation             8.805970   91.194030
Student           Business            71.266003   28.733997
                  Car                 72.899506   27.100494
                  Debt consolidation  74.026176   25.973824
                  Education           76.988984   23.011016
                  Home                67.481403   32.518597
                  Medical             77.514793   22.485207
                  Other               74.557709   25.442291
                  Vacation            75.000000   25.000000
Unemployed        Business            91.955408    8.044592
                  Car                 91.997365    8.002635
                  Debt consolidation  92.367691    7.632309
                  Education           93.275335    6.724665
                  Home                90.629880    9.370120
                  Medical             93.140930    6.859070
                  Other               91.883886    8.116114
                  Vacation            91.891892    8.108108

Cross Tabulation between Employment status and Repayment Status

In [20]:
pd.crosstab(index=[df['employment_status'], df['grade_subgrade']], columns=df['loan_paid_back'])

loan_paid_back                    0.0   1.0
employment_status grade_subgrade           
Employed          A1               21  1271
                  A2               28  1619
                  A3               23  1662
                  A4               15  1387
                  A5               41  1952
...                               ...   ...
Unemployed        F1              820     8
                  F2              730     2
                  F3              741     3
                  F4              680     5
                  F5              824     6

[150 rows x 2 columns]

In [25]:
pd.crosstab(index=[df['employment_status'], df['grade_subgrade']], columns=df['loan_paid_back'], normalize='index')*100

loan_paid_back                          0.0        1.0
employment_status grade_subgrade                      
Employed          A1               1.625387  98.374613
                  A2               1.700061  98.299939
                  A3               1.364985  98.635015
                  A4               1.069900  98.930100
                  A5               2.057200  97.942800
...                                     ...        ...
Unemployed        F1              99.033816   0.966184
                  F2              99.726776   0.273224
                  F3              99.596774   0.403226
                  F4              99.270073   0.729927
                  F5              99.277108   0.722892

[150 rows x 2 columns]

In [22]:
pd.crosstab(df['grade_subgrade'], df['loan_purpose'])

loan_purpose,Business,Car,Debt consolidation,Education,Home,Medical,Other,Vacation
grade_subgrade,,,,,,,,
A1,94,137,872,80,142,59,187,29
A2,135,191,1102,105,171,54,230,30
A3,138,198,1109,117,177,76,221,30
A4,92,180,960,85,128,47,182,27
A5,134,232,1358,109,191,87,319,41
B1,867,1477,7805,822,1133,515,1510,215
B2,981,1506,8310,864,1209,525,1587,185
B3,904,1373,7621,809,1073,495,1465,186
B4,899,1353,7685,814,1007,522,1421,176


Reference between Education Status and Loan Repayment 

In [29]:
pd.crosstab(df['education_level'], df['loan_paid_back'], normalize='index') * 100

loan_paid_back,0.0,1.0
education_level,,
Bachelor's,21.110777,78.889223
High School,19.030241,80.969759
Master's,19.765406,80.234594
Other,19.721108,80.278892
PhD,16.993286,83.006714


Reference between Marriage Status and Loan Repayment 

In [30]:
pd.crosstab(df['marital_status'], df['loan_paid_back'], normalize='index') * 100

loan_paid_back,0.0,1.0
marital_status,,
Divorced,20.335961,79.664039
Married,20.085558,79.914442
Single,20.112656,79.887344
Widowed,21.015152,78.984848


Reference between Gender and Loan Repayment 

In [31]:
pd.crosstab(df['gender'], df['loan_paid_back'], normalize='index') * 100

loan_paid_back,0.0,1.0
gender,,
Female,19.829183,80.170817
Male,20.424793,79.575207
Other,20.466738,79.533262


In [34]:
print(df.groupby('loan_paid_back')['interest_rate'].describe())

                   count       mean       std   min    25%     50%    75%  \
loan_paid_back                                                              
0.0             119500.0  12.881495  1.981273  3.89  11.55  12.875  14.17   
1.0             474494.0  12.224088  1.994191  3.20  10.87  12.250  13.53   

                  max  
loan_paid_back         
0.0             20.99  
1.0             20.84  


In [35]:
df.groupby('loan_paid_back')['loan_amount'].describe()

,count,mean,std,min,25%,50%,75%,max
loan_paid_back,,,,,,,,
0.0,119500.0,15072.223883,6945.374637,507.42,10446.6675,15078.79,19100.740,48959.95
1.0,474494.0,15007.220145,6921.722581,500.09,10235.4500,14957.85,18780.365,48954.70


In [36]:
df.groupby('loan_paid_back')['debt_to_income_ratio'].describe()

,count,mean,std,min,25%,50%,75%,max
loan_paid_back,,,,,,,,
0.0,119500.0,0.166564,0.083245,0.011,0.093,0.165,0.220,0.627
1.0,474494.0,0.109144,0.058975,0.011,0.070,0.092,0.137,0.577


In [37]:
df.groupby('loan_paid_back')['credit_score'].describe()

,count,mean,std,min,25%,50%,75%,max
loan_paid_back,,,,,,,,
0.0,119500.0,655.010628,51.680938,395.0,623.0,657.0,690.0,849.0
1.0,474494.0,687.440208,54.418255,395.0,654.0,688.0,725.0,849.0


Correlation Matrix

In [38]:
numeric_cols = ['annual_income', 'debt_to_income_ratio', 'credit_score', 
                'loan_amount', 'interest_rate', 'loan_paid_back']
print(df[numeric_cols].corr()['loan_paid_back'].sort_values())

debt_to_income_ratio   -0.335680
interest_rate          -0.131184
loan_amount            -0.003762
annual_income           0.006326
credit_score            0.234560
loan_paid_back          1.000000
Name: loan_paid_back, dtype: float64


In [40]:
print("\nGrade vs Credit Score:")
print(df.groupby('grade_subgrade')['credit_score'].mean())

print("\nGrade vs Debt-to-Income:")
print(df.groupby('grade_subgrade')['debt_to_income_ratio'].mean())

print("\nGrade vs Interest Rate:")
print(df.groupby('grade_subgrade')['interest_rate'].mean())


Grade vs Credit Score:
grade_subgrade
A1    816.643125
A2    816.397919
A3    816.264763
A4    818.126396
A5    816.197491
B1    759.075781
B2    758.985759
B3    758.907152
B4    760.068242
B5    760.343618
C1    702.838802
C2    701.498521
C3    701.070841
C4    701.600604
C5    701.468518
D1    644.665100
D2    644.426841
D3    645.201395
D4    644.839644
D5    644.892496
E1    594.090988
E2    593.711551
E3    594.301201
E4    594.634520
E5    593.584320
F1    557.985905
F2    558.252354
F3    559.597599
F4    557.910569
F5    559.496889
Name: credit_score, dtype: float64

Grade vs Debt-to-Income:
grade_subgrade
A1    0.118228
A2    0.117615
A3    0.117566
A4    0.118827
A5    0.122210
B1    0.116265
B2    0.115355
B3    0.113508
B4    0.113969
B5    0.113408
C1    0.117466
C2    0.118114
C3    0.118743
C4    0.118040
C5    0.118496
D1    0.123422
D2    0.123249
D3    0.125578
D4    0.124643
D5    0.124829
E1    0.130326
E2    0.127635
E3    0.129800
E4    0.128657
E5    0.129329


After a Lengthy EDA let'sbuild the Model

In [41]:
features = ['credit_score',
    'debt_to_income_ratio',
    'employment_status',
    'loan_purpose',
    'grade_subgrade']

In [43]:
X = df[features]
y = df['loan_paid_back']

In [44]:
print(f"Features: {X.shape}")
print(f"Target: {y.shape}")
print(f"\nClass balance in full data:")
print(y.value_counts(normalize=True))

Features: (593994, 5)
Target: (593994,)

Class balance in full data:
loan_paid_back
1.0    0.79882
0.0    0.20118
Name: proportion, dtype: float64


In [45]:
X_encoded = pd.get_dummies(X, columns=['employment_status', 'loan_purpose'],
    drop_first=True)

In [46]:
grade_mapping = {}
grades = ['A', 'B', 'C', 'D', 'E', 'F']
for i, grade in enumerate(grades):
    for subgrade in range(1, 6):
        grade_mapping[f'{grade}{subgrade}'] = i * 5 + subgrade

X_encoded['grade_numeric'] = X_encoded['grade_subgrade'].map(grade_mapping)
X_encoded = X_encoded.drop('grade_subgrade', axis=1)


In [47]:
X_encoded.head()

,credit_score,debt_to_income_ratio,employment_status_Retired,employment_status_Self-employed,employment_status_Student,employment_status_Unemployed,loan_purpose_Car,loan_purpose_Debt consolidation,loan_purpose_Education,loan_purpose_Home,loan_purpose_Medical,loan_purpose_Other,loan_purpose_Vacation,grade_numeric
0,736,0.084,False,True,False,False,False,False,False,False,False,True,False,13
1,636,0.166,False,False,False,False,False,True,False,False,False,False,False,18
2,694,0.097,False,False,False,False,False,True,False,False,False,False,False,15
3,533,0.065,False,False,False,False,False,True,False,False,False,False,False,26
4,665,0.053,False,False,False,False,False,False,False,False,False,True,False,16


In [49]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, 
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [54]:
print(f"\nTrain class balance:")
print(y_train.value_counts(normalize=True))
print(f"\nTest class balance:")
print(y_test.value_counts(normalize=True))


Train class balance:
loan_paid_back
1.0    0.798819
0.0    0.201181
Name: proportion, dtype: float64

Test class balance:
loan_paid_back
1.0    0.79882
0.0    0.20118
Name: proportion, dtype: float64


In [51]:
lr = LogisticRegression(max_iter=10000, random_state=42)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=10000, random_state=42)

In [52]:
y_pred = lr.predict(X_test)
y_proba = lr.predict_proba(X_test)[:, 1]

In [53]:
print("BASELINE MODEL PERFORMANCE")
print("="*50)
print(classification_report(y_test, y_pred))
print(f"\nROC-AUC Score: {roc_auc_score(y_test, y_proba):.4f}")
print(f"\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

BASELINE MODEL PERFORMANCE
              precision    recall  f1-score   support

         0.0       0.85      0.61      0.71     23900
         1.0       0.91      0.97      0.94     94899

    accuracy                           0.90    118799
   macro avg       0.88      0.79      0.82    118799
weighted avg       0.90      0.90      0.89    118799


ROC-AUC Score: 0.9098

Confusion Matrix:
[[14554  9346]
 [ 2553 92346]]


In [56]:
# Try different thresholds
for threshold in [0.5, 0.6, 0.7, 0.8, 0.9]:
    y_pred_thresh = (y_proba >= threshold).astype(int)
    
    print(f"\n{'='*50}")
    print(f"Threshold: {threshold}")
    print(f"{'='*50}")
    print(classification_report(y_test, y_pred_thresh))
    
    # Show business impact
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_thresh).ravel()
    print(f"\nBusiness Impact:")
    print(f"Bad loans approved (FP): {fp:,} → ${fp * 15000:,} loss")
    print(f"Good customers rejected (FN): {fn:,} → ${fn * 500:,} loss")
    print(f"TOTAL LOSS: ${fp * 15000 + fn * 500:,}")


Threshold: 0.5
              precision    recall  f1-score   support

         0.0       0.85      0.61      0.71     23900
         1.0       0.91      0.97      0.94     94899

    accuracy                           0.90    118799
   macro avg       0.88      0.79      0.82    118799
weighted avg       0.90      0.90      0.89    118799


Business Impact:
Bad loans approved (FP): 9,346 → $140,190,000 loss
Good customers rejected (FN): 2,553 → $1,276,500 loss
TOTAL LOSS: $141,466,500

Threshold: 0.6
              precision    recall  f1-score   support

         0.0       0.80      0.65      0.72     23900
         1.0       0.91      0.96      0.94     94899

    accuracy                           0.90    118799
   macro avg       0.86      0.80      0.83    118799
weighted avg       0.89      0.90      0.89    118799


Business Impact:
Bad loans approved (FP): 8,483 → $127,245,000 loss
Good customers rejected (FN): 3,768 → $1,884,000 loss
TOTAL LOSS: $129,129,000

Threshold: 0.7
  

Dropping Column grade_subgrade

In [57]:
features_no_leakage = [
    'credit_score',
    'debt_to_income_ratio',
    'employment_status',
    'loan_purpose'
]

X2 = df[features_no_leakage]
X2_encoded = pd.get_dummies(X2, columns=['employment_status', 'loan_purpose'], drop_first=True)

X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X2_encoded, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

lr2 = LogisticRegression(max_iter=10000, random_state=42)
lr2.fit(X_train2, y_train2)

y_pred2 = lr2.predict(X_test2)
y_proba2 = lr2.predict_proba(X_test2)[:, 1]

print("\n" + "="*50)
print("MODEL WITHOUT LEAKAGE")
print("="*50)
print(classification_report(y_test2, y_pred2))
print(f"ROC-AUC: {roc_auc_score(y_test2, y_proba2):.4f}")

print("\n" + "="*50)
print("COMPARISON")
print("="*50)
print(f"With grade_subgrade:    90% accuracy, 0.91 ROC-AUC")
print(f"Without grade_subgrade: ???")


MODEL WITHOUT LEAKAGE
              precision    recall  f1-score   support

         0.0       0.85      0.61      0.71     23900
         1.0       0.91      0.97      0.94     94899

    accuracy                           0.90    118799
   macro avg       0.88      0.79      0.82    118799
weighted avg       0.90      0.90      0.89    118799

ROC-AUC: 0.9099

COMPARISON
With grade_subgrade:    90% accuracy, 0.91 ROC-AUC
Without grade_subgrade: ???


In [59]:
X2_encoded.head()

,credit_score,debt_to_income_ratio,employment_status_Retired,employment_status_Self-employed,employment_status_Student,employment_status_Unemployed,loan_purpose_Car,loan_purpose_Debt consolidation,loan_purpose_Education,loan_purpose_Home,loan_purpose_Medical,loan_purpose_Other,loan_purpose_Vacation
0,736,0.084,False,True,False,False,False,False,False,False,False,True,False
1,636,0.166,False,False,False,False,False,True,False,False,False,False,False
2,694,0.097,False,False,False,False,False,True,False,False,False,False,False
3,533,0.065,False,False,False,False,False,True,False,False,False,False,False
4,665,0.053,False,False,False,False,False,False,False,False,False,True,False


In [60]:
print("="*50)
print("MODEL 1 (With Leakage) - Actual Features Used:")
print("="*50)
print(X_encoded.columns.tolist())
print(f"Total features: {len(X_encoded.columns)}")

print("\n" + "="*50)
print("MODEL 2 (Without Leakage) - Actual Features Used:")
print("="*50)
print(X2_encoded.columns.tolist())
print(f"Total features: {len(X2_encoded.columns)}")

print("\n" + "="*50)
print("DIFFERENCE:")
print("="*50)
diff = set(X_encoded.columns) - set(X2_encoded.columns)
print(f"Features only in Model 1: {diff}")

MODEL 1 (With Leakage) - Actual Features Used:
['credit_score', 'debt_to_income_ratio', 'employment_status_Retired', 'employment_status_Self-employed', 'employment_status_Student', 'employment_status_Unemployed', 'loan_purpose_Car', 'loan_purpose_Debt consolidation', 'loan_purpose_Education', 'loan_purpose_Home', 'loan_purpose_Medical', 'loan_purpose_Other', 'loan_purpose_Vacation', 'grade_numeric']
Total features: 14

MODEL 2 (Without Leakage) - Actual Features Used:
['credit_score', 'debt_to_income_ratio', 'employment_status_Retired', 'employment_status_Self-employed', 'employment_status_Student', 'employment_status_Unemployed', 'loan_purpose_Car', 'loan_purpose_Debt consolidation', 'loan_purpose_Education', 'loan_purpose_Home', 'loan_purpose_Medical', 'loan_purpose_Other', 'loan_purpose_Vacation']
Total features: 13

DIFFERENCE:
Features only in Model 1: {'grade_numeric'}


In [71]:
# Create LightGBM dataset
train_data = lgb.Dataset(X_train2, label=y_train2)
test_data = lgb.Dataset(X_test2, label=y_test2, reference=train_data)

# Parameters
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'verbose': -1
}

# Train
print("Training LightGBM...")
lgb_model = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[test_data],
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

# Predict
y_pred_lgb = lgb_model.predict(X_test2)
y_pred_lgb_binary = (y_pred_lgb >= 0.5).astype(int)

# Evaluate
print("\n" + "="*50)
print("LIGHTGBM PERFORMANCE")
print("="*50)
print(classification_report(y_test2, y_pred_lgb_binary))
print(f"ROC-AUC: {roc_auc_score(y_test2, y_pred_lgb):.4f}")

Training LightGBM...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[723]	valid_0's auc: 0.921191

LIGHTGBM PERFORMANCE
              precision    recall  f1-score   support

         0.0       0.88      0.61      0.72     23900
         1.0       0.91      0.98      0.94     94899

    accuracy                           0.91    118799
   macro avg       0.90      0.80      0.83    118799
weighted avg       0.90      0.91      0.90    118799

ROC-AUC: 0.9212


In [72]:
from sklearn.metrics import confusion_matrix

# Get confusion matrix
cm = confusion_matrix(y_test2, y_pred_lgb_binary)
tn, fp, fn, tp = cm.ravel()

print("Confusion Matrix:")
print(f"True Negatives (correctly rejected): {tn:,}")
print(f"False Positives (bad loans approved): {fp:,}")
print(f"False Negatives (good customers rejected): {fn:,}")
print(f"True Positives (good loans approved): {tp:,}")

print(f"\nBusiness Impact (Threshold 0.5):")
print(f"Bad loans approved: {fp:,} × $15,000 = ${fp * 15000:,}")
print(f"Good customers rejected: {fn:,} × $500 = ${fn * 500:,}")
print(f"TOTAL LOSS: ${fp * 15000 + fn * 500:,}")

print("\nCompare to Logistic Regression:")
print("LR Total Loss: $141,466,500")

Confusion Matrix:
True Negatives (correctly rejected): 14,661
False Positives (bad loans approved): 9,239
False Negatives (good customers rejected): 1,971
True Positives (good loans approved): 92,928

Business Impact (Threshold 0.5):
Bad loans approved: 9,239 × $15,000 = $138,585,000
Good customers rejected: 1,971 × $500 = $985,500
TOTAL LOSS: $139,570,500

Compare to Logistic Regression:
LR Total Loss: $141,466,500


In [73]:
print("\n" + "="*50)
print("THRESHOLD OPTIMIZATION (LightGBM)")
print("="*50)

for threshold in [0.5, 0.6, 0.7, 0.8]:
    y_pred_thresh = (y_pred_lgb >= threshold).astype(int)
    
    tn, fp, fn, tp = confusion_matrix(y_test2, y_pred_thresh).ravel()
    total_loss = fp * 15000 + fn * 500
    
    print(f"\nThreshold: {threshold}")
    print(f"  Accuracy: {(tn + tp) / len(y_test2):.3f}")
    print(f"  Precision (Class 0): {tn / (tn + fn):.3f}")
    print(f"  Recall (Class 0): {tn / (tn + fp):.3f}")
    print(f"  Bad loans: {fp:,} → ${fp * 15000:,}")
    print(f"  Lost customers: {fn:,} → ${fn * 500:,}")
    print(f"  TOTAL LOSS: ${total_loss:,}")


THRESHOLD OPTIMIZATION (LightGBM)

Threshold: 0.5
  Accuracy: 0.906
  Precision (Class 0): 0.881
  Recall (Class 0): 0.613
  Bad loans: 9,239 → $138,585,000
  Lost customers: 1,971 → $985,500
  TOTAL LOSS: $139,570,500

Threshold: 0.6
  Accuracy: 0.903
  Precision (Class 0): 0.814
  Recall (Class 0): 0.672
  Bad loans: 7,843 → $117,645,000
  Lost customers: 3,661 → $1,830,500
  TOTAL LOSS: $119,475,500

Threshold: 0.7
  Accuracy: 0.894
  Precision (Class 0): 0.738
  Recall (Class 0): 0.731
  Bad loans: 6,426 → $96,390,000
  Lost customers: 6,207 → $3,103,500
  TOTAL LOSS: $99,493,500

Threshold: 0.8
  Accuracy: 0.866
  Precision (Class 0): 0.634
  Recall (Class 0): 0.793
  Bad loans: 4,939 → $74,085,000
  Lost customers: 10,947 → $5,473,500
  TOTAL LOSS: $79,558,500


In [74]:
# Create new features
df['income_to_loan_ratio'] = df['annual_income'] / df['loan_amount']
df['credit_times_income'] = df['credit_score'] * df['annual_income'] / 100000
df['debt_times_rate'] = df['debt_to_income_ratio'] * df['interest_rate']

# Add to your feature list
features_engineered = features_no_leakage + [
    'income_to_loan_ratio',
    'credit_times_income', 
    'debt_times_rate'
]

In [75]:
# Get LightGBM feature importance
importance = pd.DataFrame({
    'feature': X_train2.columns,
    'importance': lgb_model.feature_importance()
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(importance)


Feature Importance:
                            feature  importance
1              debt_to_income_ratio       10143
0                      credit_score        7901
7   loan_purpose_Debt consolidation         511
5      employment_status_Unemployed         469
4         employment_status_Student         441
11               loan_purpose_Other         366
6                  loan_purpose_Car         323
3   employment_status_Self-employed         320
8            loan_purpose_Education         288
9                 loan_purpose_Home         283
2         employment_status_Retired         282
10             loan_purpose_Medical         225
12            loan_purpose_Vacation         138


In [76]:
# Create powerful new features
df['income_to_loan_ratio'] = df['annual_income'] / (df['loan_amount'] + 1)  # +1 to avoid division by zero
df['credit_to_debt_ratio'] = df['credit_score'] * (1 - df['debt_to_income_ratio'])
df['risk_score'] = df['debt_to_income_ratio'] * df['interest_rate'] / df['credit_score']

# Advanced: Employment + Credit Score interaction
# Students with low credit = VERY risky
# Retired with high credit = VERY safe
df['is_high_risk'] = ((df['employment_status'].isin(['Student', 'Unemployed'])) & 
                       (df['credit_score'] < 650)).astype(int)

df['is_low_risk'] = ((df['employment_status'].isin(['Retired', 'Employed'])) & 
                      (df['credit_score'] > 700)).astype(int)

In [77]:
# Update feature list
features_v2 = [
    'credit_score',
    'debt_to_income_ratio',
    'employment_status',
    'loan_purpose',
    # NEW FEATURES
    'income_to_loan_ratio',
    'credit_to_debt_ratio',
    'risk_score',
    'is_high_risk',
    'is_low_risk'
]

# Prepare data
X_v2 = df[features_v2]
X_v2_encoded = pd.get_dummies(X_v2, columns=['employment_status', 'loan_purpose'], drop_first=True)

# Train/test split
X_train_v2, X_test_v2, y_train_v2, y_test_v2 = train_test_split(
    X_v2_encoded, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Train LightGBM
train_v2 = lgb.Dataset(X_train_v2, label=y_train_v2)
test_v2 = lgb.Dataset(X_test_v2, label=y_test_v2, reference=train_v2)

print("Training LightGBM v2 with engineered features...")
lgb_v2 = lgb.train(
    params,
    train_v2,
    num_boost_round=1000,
    valid_sets=[test_v2],
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

# Evaluate
y_pred_v2 = lgb_v2.predict(X_test_v2)
y_pred_v2_binary = (y_pred_v2 >= 0.5).astype(int)

print("\n" + "="*50)
print("IMPROVED MODEL WITH FEATURE ENGINEERING")
print("="*50)
print(classification_report(y_test_v2, y_pred_v2_binary))
print(f"ROC-AUC: {roc_auc_score(y_test_v2, y_pred_v2):.4f}")

print("\nComparison:")
print(f"Baseline LightGBM: 91.0% accuracy, 0.9212 AUC")
print(f"With new features: ???")

Training LightGBM v2 with engineered features...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[525]	valid_0's auc: 0.920505

IMPROVED MODEL WITH FEATURE ENGINEERING
              precision    recall  f1-score   support

         0.0       0.89      0.61      0.72     23900
         1.0       0.91      0.98      0.94     94899

    accuracy                           0.91    118799
   macro avg       0.90      0.79      0.83    118799
weighted avg       0.90      0.91      0.90    118799

ROC-AUC: 0.9205

Comparison:
Baseline LightGBM: 91.0% accuracy, 0.9212 AUC
With new features: ???


In [79]:
df_test = pd.read_csv("bank_test.csv")

print(f"Test data loaded: {len(df_test)} rows")
print(f"\nTest columns:")
print(df_test.columns.tolist())
print(f"\nFirst few rows:")
print(df_test.head())

Test data loaded: 254569 rows

Test columns:
['id', 'annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate', 'gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']

First few rows:
       id  annual_income  debt_to_income_ratio  credit_score  loan_amount  \
0  593994       28781.05                 0.049           626     11461.42   
1  593995       46626.39                 0.093           732     15492.25   
2  593996       54954.89                 0.367           611      3796.41   
3  593997       25644.63                 0.110           671      6574.30   
4  593998       25169.64                 0.081           688     17696.89   

   interest_rate  gender marital_status education_level employment_status  \
0          14.73  Female         Single     High School          Employed   
1          12.85  Female        Married        Master's          Employed   
2          13.29    Male         Single      B

In [80]:
# Select same features as training
features_final = [
    'credit_score',
    'debt_to_income_ratio',
    'employment_status',
    'loan_purpose'
]

X_submit = df_test[features_final]

# One-hot encode (SAME as training!)
X_submit_encoded = pd.get_dummies(
    X_submit,
    columns=['employment_status', 'loan_purpose'],
    drop_first=True
)

print(f"\nTest features after encoding: {X_submit_encoded.shape}")
print(f"Feature names: {X_submit_encoded.columns.tolist()}")

# CRITICAL: Make sure columns match training data!
# Add missing columns with 0, remove extra columns
missing_cols = set(X_train2.columns) - set(X_submit_encoded.columns)
for col in missing_cols:
    X_submit_encoded[col] = 0
    
# Reorder columns to match training
X_submit_encoded = X_submit_encoded[X_train2.columns]

print(f"\nAfter alignment: {X_submit_encoded.shape}")
print(f"Matches training shape? {X_submit_encoded.shape[1] == X_train2.shape[1]}")


Test features after encoding: (254569, 13)
Feature names: ['credit_score', 'debt_to_income_ratio', 'employment_status_Retired', 'employment_status_Self-employed', 'employment_status_Student', 'employment_status_Unemployed', 'loan_purpose_Car', 'loan_purpose_Debt consolidation', 'loan_purpose_Education', 'loan_purpose_Home', 'loan_purpose_Medical', 'loan_purpose_Other', 'loan_purpose_Vacation']

After alignment: (254569, 13)
Matches training shape? True


In [81]:
# Predict probabilities
y_submit_proba = lgb_model.predict(X_submit_encoded)

# Apply threshold 0.7 (our recommended threshold!)
threshold = 0.7
y_submit_pred = (y_submit_proba >= threshold).astype(int)

print(f"\nPredictions made with threshold {threshold}")
print(f"Predicted paid back: {(y_submit_pred == 1).sum():,}")
print(f"Predicted default: {(y_submit_pred == 0).sum():,}")
print(f"Percentage approved: {(y_submit_pred == 1).mean() * 100:.1f}%")


Predictions made with threshold 0.7
Predicted paid back: 204,370
Predicted default: 50,199
Percentage approved: 80.3%


In [84]:
# Create submission with PROBABILITIES (not binary!)
submission_v2 = pd.DataFrame({
    'id': df_test['id'],
    'loan_paid_back': y_submit_proba  # ← Use probabilities!
})

# Verify it looks right
print("First 10 rows (should be decimals, not 0/1):")
print(submission_v2.head(10))

# Save
submission_v2.to_csv('submission_probabilities.csv', index=False)
print(f"\n✅ NEW submission created with probabilities!")

First 10 rows (should be decimals, not 0/1):
       id  loan_paid_back
0  593994        0.898371
1  593995        0.983658
2  593996        0.526755
3  593997        0.906039
4  593998        0.944163
5  593999        0.976405
6  594000        0.984068
7  594001        0.976161
8  594002        0.961308
9  594003        0.002458

✅ NEW submission created with probabilities!


In [83]:
# Compare train vs test predictions
print("Training set approval rate:")
y_train_pred = (lgb_model.predict(X_train2) >= 0.7).astype(int)
print(f"  Approved: {(y_train_pred == 1).mean() * 100:.1f}%")

print("\nTest set approval rate:")
print(f"  Approved: {(y_submit_pred == 1).mean() * 100:.1f}%")

# These should be similar! If train=80% and test=40%, something is wrong

Training set approval rate:
  Approved: 80.2%

Test set approval rate:
  Approved: 80.3%


Had a rank of 848 but let's see if it can be improved

In [86]:
pip install catboost


   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.8/102.4 MB 4.9 MB/s eta 0:00:21
    --------------------------------------- 2.4/102.4 MB 6.5 MB/s eta 0:00:16
   - -------------------------------------- 4.2/102.4 MB 7.4 MB/s eta 0:00:14
   -- ------------------------------------- 6.6/102.4 MB 8.2 MB/s eta 0:00:12
   --- ------------------------------------ 8.7/102.4 MB 8.7 MB/s eta 0:00:11
   ---- ----------------------------------- 11.5/102.4 MB 9.4 MB/s eta 0:00:10
   ----- ---------------------------------- 14.2/102.4 MB 9.8 MB/s eta 0:00:09
   ------ --------------------------------- 16.5/102.4 MB 10.1 MB/s eta 0:00:09
   ------- -------------------------------- 19.4/102.4 MB 10.4 MB/s eta 0:00:08
   -------- ------------------------------- 21.8/102.4 MB 10.6 MB/s eta 0:00:08
   --------- ------------------------------ 24.9/102.4 MB 10.8 MB/s eta 0:00:08
   ---------- ----------------------------- 26.2/102.4 MB 10

In [87]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# XGBoost
xgb = XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=8, random_state=42)
xgb.fit(X_train, y_train)
xgb_proba = xgb.predict_proba(X_test)[:, 1]

# CatBoost  
cat = CatBoostClassifier(iterations=500, learning_rate=0.05, depth=8, verbose=0, random_state=42)
cat.fit(X_train, y_train)
cat_proba = cat.predict_proba(X_test)[:, 1]

# Ensemble (weighted average)
ensemble_proba = (lgb_proba * 0.4 + xgb_proba * 0.3 + cat_proba * 0.3)

# Evaluate
print(f"LightGBM: {roc_auc_score(y_test, lgb_proba):.4f}")
print(f"XGBoost: {roc_auc_score(y_test, xgb_proba):.4f}")
print(f"CatBoost: {roc_auc_score(y_test, cat_proba):.4f}")
print(f"Ensemble: {roc_auc_score(y_test, ensemble_proba):.4f}")

NameError: name 'lgb_proba' is not defined